# Pairwise Sequence Alignemt
---------------------------------------

Before you begin this exercise, please read [Durbin Chapter 2: Sections 2.1-2.3, 2.7,2.8 ](https://github.com/uchicago-scientific/cdn/raw/main/module-3-assets/Durbin-Chapter2.pdf) on pairwise sequence alignment.

# Alignment Matters
---------------------------------------

In written text, spaces matter a lot. For example consider the following nonsense phrase:

>HIDING OLD PANTHERS WORDS ARE SHARP 

It describes the mean words of a hiding feline. Now compare against this one:

>HID IN GOLD PANT HER SWORDS ARE SHARP

It describes the sharpness of swords in someone's pants.

The only difference between the two phrases is the placement of spaces - all the letters are otherwise the same. (You can take a moment to check this - I worked really hard coming up with these ;))

Now let's think about an evolving DNA sequence. That sequence can undergo several types of mutations. These include substitutions, where one nucleotide in the DNA  is replaced with another (such as A --> T), but can also include insertions - in which a letter is added in the middle of a sequence (such as 'A' -> 'AT')  - or deletions (like 'AT' -> 'A'). 

Much like the nonsense phrases above, insertions or deletions (collectively known as 'indels') can cause *misalignment* between the letters of an ancestral DNA sequence and its descendants. The process of *sequence alignment* attempts to detect where insertions or deletions have occurred in order to more accurately line up nucleotides that share an evolutionary origin.

# Flavors of Sequence Alignment: Global vs. Local Alignment 

Sequence alignment comes in several different flavors or varieties:

**Global sequence alignment** assumes two sequences are homologous (or share a common evolutionary ancestor) across their whole length. It tries to line up the whole thing, from tail to snout. This might be a reasonable assumption for two genes in the same gene family, for instance. 

**Local sequence alignment** assumes that there are regions of homology within a bigger sequence, but also regions that do not share common ancestry. This might be a reasonable assumption if checking how many genes two species share. If the species aren't extremely closely related, we would expect that they would share certain genes, but also that there should be large portions of the genome that do not share detectable evolutionary ancestry between the two species (due to large insertions, deletions, inversions or rearrangements). 

Alignment methods can also be **pairwise** methods, in which two sequences are aligned or **multiple sequence alignments** in which many sequences are aligned together (e.g. all the amino-acyl tRNA synthetase genes in mammals).

In this chapter we will focus first on global alignment. However, once you understand global alignment in detail, the algorithm for local alignment is similar enough that you can understand that as well, with just a little extra effort. 

In **pairwise global sequence alignment** we begin with two sequences. These can be DNA, RNA or amino acid sequences. For simplicity, let's focus at first on pairwise global alignment of DNA sequences. 

We'll assume that the full length of our two sequences are homologous, but may contain *insertions* or *deletions* of nucleotides that have caused the homologous positions not to line up with each other. Therefore, our goal is to 'align' or line up these homologous positions in the sequences by adding '-' characters to certain places in each sequence.  These '-' characters will represent places where the two nucleotide sequences become offset from one another (and therefore stop lining up) due to either an extra nucleotide inserted into one, or a nucleotide deleted from the other. 

Here's an example. Let's start with two unaligned DNA sequences: `AAAATATTGG` and `TCCTATGG`. If we try to compare sequence identity between these sequences without alignment, we won't see a high level of sequence similarity:

```
AAAATATTGG
TCCTATGG
```

Indeed, if you compare the nulceotides that line up, you'll see that *none* of them match. If we wanted to formalize this, we could calculate the Hamming Distance between the two sequences (just 1 - the fraction of matching items), and we would find a distance of 1.0.

Yet it is probably clear that the sequences have many similarities. If we upload these sequences to a pairwise global alignment web tool (like the EMBOSS one at [EBI](https://www.ebi.ac.uk/Tools/services/web/toolresult.ebi?jobId=emboss_needle-I20201212-184827-0832-96343451-p1m)), we might get a result that looks something like this:

```
EMBOSS_001         1 AAAATAT----TGG     10
                           |    |||
EMBOSS_001         1 ------TCCTATGG      8
```

From a pairswise alignment alone, we can't distinguish whether '-' characters represent insertion in the sequence without the '-' or deletion in the sequence with the '-'. For this reason '-' characters are called *gaps* or *indels* (short for *in*sertion **or** *del*etion).

# Global Sequence Alignment with Needleman-Wunsch

### Understanding the Needleman-Wunsch algorithm

----


Before we write code to accomplish a task, it's critical that we understand exactly what we are trying to do. For most algorithms, that means it is extremely important that we work a couple examples by hand before we try to code them. This might seem like extra work, but in fact saves **a ton** of time in the long run. 

Therefore, before we try to write code to do Needleman-Wunsch alignment, we're going to practice doing it by hand. Don't worry - after the first row or two it will become clear - even obvious - how to do the algorithm. 

> **Coding tip for complex code:** If you take the time to really experiment with each python data structure and understand how it works, and take the time to do an algorithm by hand so you understand how it works, and if you build your code step by step in small pieces that you can check as you go, then even very complex algorithms become much easier to put together.

### Performing the Needleman-Wunsch algorithm by hand

--------------------

Grab a piece of paper and a pen. We're going to read the Needleman-Wunsch algorithm description and implement it for a simple example on paper. 

### Settle on a scoring scheme
First, we need to settle on a scoring scheme that is based on how much we want to penalize insertion of indels ('-') vs. substitions (e.g. A --> G). We can choose to score all substitutions equally, or assign a different score to some substitutions (e.g. to reflect that biologically transitions are much more common than transversions). Generally, the more common insertions or deletions are biologically, the lower the gap penalty should be. Similarly, the more common substitutions are, the lower the substitution penalty should be. For simplicity in this example, we will use a very simple scoring scheme:

- *Gap penalty* = -1 
- *Match bonus (all matches)* = 1 
- *Mismatch penalty (all substitutions)* = -1 


### Draw a table representing possible alignments
Next, we need to draw two tables in which the nucleotides of sequence 1 are on the left side (i.e. they are the row labels) and the nucleotides of sequence 2 are on the top (i.e. they are the column labels). The first will be our 'score matrix' where we keep track of the best possible way to get to the point where each pair of nucletides line up. The second will be our traceback matrix, where we record the set of steps that got us to that optimal score. One trick is that we will have an extra gap ('-') character before each sequence (so the number of rows be one more than the length of sequence 1, and the number of columns will be one more than the length of sequence 2. At the end we will be able to walk back through the traceback matrix to get our optimal alignment.

Here's an example of how the scoring matrix might look at the start of the algorithm:

![](https://github.com/uchicago-scientific/cdn/raw/main/needleman_wunsch_alignment_step2_colored.jpg)

_The image shows a table formed from the DNA sequence G C A T G C T on the x-axis (each column corresponds to one letter) and the sequence G A T T A C A on the y-axis. There is one extra blank row or column before each sequence. At this step, the table is blank. The two sequences are rainbow colored, with each nucleotide being a different color._

### Intuition behind the layout of the scoring matrix

One key insight of the Needleman-Wunsch alignment is that all the possible alignments of two sequences from left to right can be represented on this table as paths between the upper left of the table and the lower right of the table. 

**Meaning of diagonal moves.** If we start in the upper left corner and proceed digonally down the table, that represents an alignment in which we only use *substitutions* (mutations that change one nucleotide into another) to explain the differences between the two sequences. 

In our specific example, this would mean an alignment that looks like this:

```
1 GCATGCT     7
   |  |
1 GATTACA     7
```

This is a pretty bad alignment (only 2 matches and 5 mismatches). We'll improve on it in a moment, but can you see how the row/column labels you would encounter if you go from top left to bottom right in the table correspond to this alignment? If not, it may be worth trying it by hand step-by-step: the first cell you'd enter in the top left would have a G in both the row and the column. That represents the match between the G's in the top and bottom sequence in the alignment above. The next step would have a C in the column but an A in the row. That would represent the mismatch between C and A in the second position of the alignment. Mistmatches like this indicate that during evolution either the ancestor was a C and one sequence had a C --> A substitution in the bottom sequence, or the ancestor was an A and there was an A--> C substitution in the top sequence. 


More generally, any diagonal move we make in heading from the start to the end of the sequence means we are asserting that two nucleotides are homologous (share the same evolutionary origin) and are 'the same thing'. This type of move will improve the alignment score if the nucleotides are the same, but make it worse if they are different.

**Meaning of moves to the right**
Let's say after starting in the upper-left of the table we took one step to the right, *and then* proceeded as before to travel diagonally down the table. What would that mean for our alignment? If you try this with your finger on the table above, you'll see that the first cell you reach has column label G and now row label. This would indicate that the first G in our first sequence is paired with nothing ('-') in our second sequence. Biologically, this would indicate that the ancestral sequence either had a G and it was lost in the bottom sequence (a deletion) or that the G was added by mutation into the top sequence (an insertion). Now move your finger down and to the right. The next cell you encounter is no longer a pairing of G with G (as before) but now a pairing of the first G in the bottom sequence with the C in position 2 of the top sequence. Effectively, all our other pairings will be 'offset' by 1 due to the insertion in the left sequence.  This will change which positions look like matches vs. mismatches. The alignment corresponding to this 'horizontal 1 then diagonal' path would look like this:

`1 GCATGCT-     8
      |
 1 -GATTACA     8
`

**Meaning of moves down**

Just as moves to the right insert indel ('-') characters into the left (rows) sequence, moves down insert indel characters into the top (columns) sequence.

> Needleman-Wunsch alignment represents all possible alignments of two sequences using a table. Any set of right, down or diagonal down and to the right moves represents a possible alignment of the two sequences. Diagonal paths represent matches or mismatches. Moves to the right insert an indel character into the second (rows) sequence, while moves down insert an indel character into the first (columns) sequence. Any move right or down changes which nucleotides match or mismatch in subsequent diagonal moves.




## Step 1. Fill in first rows and columns

----

Our first step will be to fill in a score of "0" in the upper left of the table. the first set of rows and columns. We said that each move to the right or down is an insertion/deletion event. 

Let's assume a 'gap penalty' of -1. This means each insertion or deletion event penalizes our score by 1 (similar to a mismatch between nucleotides). The score of each cell will be the score of the best path(s) to get to that cell.  

![](https://github.com/uchicago-scientific/cdn/raw/main/needleman_wunsch_alignment_step3_colored.jpg)

_The image shows a table formed from the DNA sequence G C A T G C T on the x-axis (each column corresponds to one letter) and the sequence G A T T A C A on the y-axis. There is one extra blank row or column before each sequence. The 1st row below the top sequence has numbers 0, -1, -2 up through -7 in the cells. The first column before the 2nd sequence also has the numbers 0, -1, -2, through -7 (from top to bottom) in the cells._

## Step 2. Score  the first cell

----

We will now fill in scores in the first row, working from left to right, following a couple of simple rules. Let's start with the first cell, with the G's on the columns and rows. We'll talk in depth about the rules for how to score this cell, then you can practice it on later cells.

### Calculate scores for the 3 possible moves that can lead to this cell

For each cell we need to calculate 3 values, representing the 3 possible moves (down, right, or diagonal) that can get us to the current cell, to figure out which has the highest score and therefore the best path through the alignment to this spot. Let's do this one by one for the first cell.

First let's calculate the score to get to the cell using a **down** move. This score will be equal to the score in the cell above plus the gap penalty. So for our first cell, the score above is -1, and the gap penalty is -1. So if we got to this spot by a down move (inserting an indel into the left sequence), the score would be -2.

A move **to the right** also represents insertion of an indel. If our top left position was reached by such a move, it would have a score of -1 (the score in the cell to the left of our current cell) + the gap penalty of -1. So it would have a score of -2. 

Finally, the score could have been produced by a move **diagonally down** (representing a match between the two G's). The scoring for diagonal moves is to begin with the score diagonally above, then add the match bonus (here +1) if the two nucleotides in the row and column for that position match, or the mismatch penalty (here -1) for those two nucleotides if they don't match. In this case they do match, so the score for this move is 0 (the score of the cell above and to the left) + the match bonus of 1 = 1. 

### Record the best scoring move
Finally, we take the highest of these 3 scores and put it into the cell. This represents the best way that previous nucleotides in the two sequences could be aligned up to the current spot of the alignment - in other words the best path to this spot. We also draw an arrow to represent the direction of the best scoring move (here diagonal down). This can either be recorded in another table that is a copy of this one, or you can just draw the arrows into the corners of the cells like I've done below.

>**The score of each cell in a Needleman Wunsch scoring matrix** is the *maximum score* produced by either:
>Reaching it with a down move: score of cell above + gap penalty
>Reacing it with a move to the right: score of cell to the left + gap penalty
>Reacing it with a diagonal move: score of the cell above and to the left of this cell, + either the match bonus (if the row and column nucleotides match) or the mismatch penalty (if they don't).


**What this result means**: In this case, the best score is produced by a diagonal (match or mismatch) move that matches the two G's for a score of 1. Biologically and intuitively this makes sense: if two sequences start with a G, a reasonable first step is to line up those G's. 


![](https://github.com/uchicago-scientific/cdn/raw/main/needleman_wunsch_alignment_step4_colored.jpg)

_The image shows a table formed from the DNA sequence G C A T G C T on the x-axis (each column corresponds to one letter) and the sequence G A T T A C A on the y-axis. There is one extra blank row or column before each sequence. The 1st row below the top sequence has numbers 0, -1, -2 up through -7 in the cells. The first column before the 2nd sequence also has the numbers 0, -1, -2, through -7 (from top to bottom) in the cells. The cell immediately diagonally down from the top left is marked '1'._




### Step 3. Repeat for the rest of the cells in the table

-----

Repeat the process from Step 2 above to score all the other cells in the table. Here's how that would look after the first row (I've omitted all the arrows to make the table a little less dense):

![](https://github.com/uchicago-scientific/cdn/raw/main/needleman_wunsch_alignment_step5_colored.jpg)

_The image shows a table formed from the DNA sequence G C A T G C T on the x-axis (each column corresponds to one letter) and the sequence G A T T A C A on the y-axis. There is one extra blank row or column before each sequence. The 1st row below the top sequence has numbers 0, -1, -2 up through -7 in the cells. The first column before the 2nd sequence also has the numbers 0, -1, -2, through -7 (from top to bottom) in the cells._

### Step 4. Trace back through the matrix to find the optimal alignment

___

Finally, start in the lower right of the alignment. Step to the cell that has the highest score and is either above, to the left or diagonal up and to the left of the current cell. Each time you step diagonally, record a match or mismatch between the row & diagonal sequences. 

Whenever you move up, record an indel in the left sequence. Whenever you move right, record an indel in the top sequence. If you move diagonally, record each nucleotide in it's respective sequence.

So for the first move (a diagonal), we'd record:

```
T
A
```

Since these are the nucleotides in the lower left.

After completing the full traceback, we would finally we would arrive at:

```
GCAT-GCT
| ||
G-ATTACA
```

![](https://github.com/uchicago-scientific/cdn/raw/main/needleman_wunsch_alignment_step6_colored.jpg)

_The image shows a table formed from the DNA sequence G C A T G C T on the x-axis (each column corresponds to one letter) and the sequence G A T T A C A on the y-axis. There is one extra blank row or column before each sequence. The 1st row below the top sequence has numbers 0, -1, -2 up through -7 in the cells. The first column before the 2nd sequence also has the numbers 0, -1, -2, through -7 (from top to bottom) in the cells._

_Exercise adapted from Full Spectrum Bioinformatics (J. Zaneveld, 2020)_